In [1]:
import pandas as pd
import pickle

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
cred = credentials.Certificate('./drone-capstone-386903-d2b13d68a9d6.json')

try:
    firebase_admin.initialize_app(cred)
except Exception as e:
    print(f"An error occurred: {str(e)}")
    
db = firestore.client()

In [3]:
users_ref = db.collection('reviews')
docs = users_ref.get()

data_list = []

for doc in docs:
    data = doc.to_dict()
    data_list.append(data)

data = pd.DataFrame(data_list)

with open('./recomender_models/final_rating.pickle', 'rb') as f:
    final_rating = pickle.load(f)

In [4]:
# Get recommendeation by calculated predicted_rating & sentiment_score
def get_recommendation(final_rating, user_id, data, no_rec):
    def getReviewData(x):
        product_sentiments = data[data['laundry_id'] == x]['sentiment_score']
        sentiment_score = product_sentiments.mean()
        return sentiment_score

    user_ratings = final_rating.loc[user_id]
    recommendation = user_ratings.sort_values(ascending=False)[:no_rec]

    recommendation_table = pd.DataFrame(recommendation).reset_index().rename(columns={user_id: 'predicted_ratings'})
    recommendation_table.insert(0, 'user_id', user_id)

    recommendation_table['sentiment_score'] = recommendation_table['laundry_id'].apply(getReviewData)
    recommendation_table['product_ranking_score'] = recommendation_table['predicted_ratings'] + recommendation_table['sentiment_score']
    sorted_recommendations = recommendation_table.sort_values(by='product_ranking_score', ascending=False).head(no_rec)
    
    return sorted_recommendations['laundry_id']

In [5]:
# Adjust with send request user_id data
user_id = "045UoknM0cMhYuk8ORyv"

recommendation_table = get_recommendation(final_rating, user_id, data, no_rec=15)
print(recommendation_table)

6     fWGrFTXnJ2DdAos7RofE
10    ggPaqhkzitso7XgHh8Zz
9     eoWGZL31kIeg10fmu6aC
14    eFIzcKy7GED8P7n6b4Gc
7     fVuqrsv1KNUMf7NZJYv2
4     fmWCFFqZcQ4KS3QFL4s9
11    eXLp4CXQCOb7sY32QfXD
2     gC0wFj8B9SnAVztqs1dH
3     gBTPcfkyJB3253EfokY5
13    eHf0KdMjSZGFBfxpsvBQ
8     fRQGTXbiSqMqFbGjrGe6
0     0KM8mgEdRPBi8zv7Mr6w
5     ff6tKtqJYXwsQboD8HYO
12    eNS8oL4kBqTUbnBQOOGb
1     fGn2GZjG1lmB0WOZAsu2
Name: laundry_id, dtype: object
